In [50]:
%load_ext autoreload
%autoreload 2

from processLIDC import Patient
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tqdm import tqdm
from yolodataset import create_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
patient_id = 'LIDC-IDRI-0002'
patient = Patient(patient_id)
patient.scale()

In [52]:
patient.get_all_nodules()

Paciente: LIDC-IDRI-0002
___________________________________
Paciente del nodulo: LIDC-IDRI-0002
Primera slice con el nodulo 171
num. slices nodulo: 28
-----------
Paciente del nodulo: LIDC-IDRI-0002
Primera slice con el nodulo 176
num. slices nodulo: 23
-----------
___________________________________


In [53]:
# patient.reconstruct_body(nodulos=True)

In [54]:
patient.imshow(slices=(53,54,55), label=True, scaled=True)

obteniendo los datos...


## Funcion para guardar las imagenes en formato .jpg

In [55]:
path2newdataset='/home/faraujo/TFM/dataset_cero_val'
# dicom2jpg(patient_id, path2newdataset)


path2olddataset = '/home/faraujo/TFM/manifest-1675801116903/LIDC-IDRI'

# create_dataset(path2olddataset, path2newdataset, val=0, percent_include=1)



predecir desde terminal:

`yolo segment predict source='/home/faraujo/TFM/dataset_cero_val/train/images/LIDC-IDRI-0129_54.png' model=/home/faraujo/TFM/yolo_trainings/train_100_include02_large/best.pt show=True conf=0.5 save=False`